# Blue Brain BioExplorer
![](../bioexplorer_banner.png)

### Prerequeries

In [ ]:
!pip install ipyTransferFunction

# Visualizing fields of magnetic forces (work in progress!)

In [ ]:
from bioexplorer import BioExplorer
from ipyTransferFunction import TransferFunctionEditor
uri = 'localhost:5000'
be = BioExplorer(uri)
be.reset()

resource_folder = '../../tests/test_files/'
colormap_folder = resource_folder + 'colormap/'

### Import fields from file

In [ ]:
filename = resource_folder + 'fields/receptor.fields'
status = be.import_fields_from_file(filename=filename)

In [ ]:
be = BioExplorer(uri)
core = be.core_api()
fields_model_id = core.scene.models[len(core.scene.models)-1]['id']

### Set Field renderer with appropriate settings

In [ ]:
image_samples_per_pixel = 1

In [ ]:
core.set_renderer(
    current='bio_explorer_fields',
    samples_per_pixel=1, subsampling=1, max_accum_frames=image_samples_per_pixel)

params = core.BioExplorerFieldsRendererParams()
params.cutoff = 15000
params.exposure = 1.0
params.alpha_correction = 1
params.nb_ray_steps = 8
params.nb_ray_refinement_steps = image_samples_per_pixel
params.use_hardware_randomizer = True
status = core.set_renderer_params(params)

### Set transfer function range

In [ ]:
def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))

def update_palette(tf):
    intensity = 1
    btf = core.get_model_transfer_function(id=fields_model_id)
    colors = list()
    points = list()
    
    nb_points = len(tf.alpha_sliders)
    step = 1.0 / float(nb_points - 1)
    for i in range(nb_points):
        color = hex_to_rgb(tf.color_pickers[i].value)
        colors.append([
            intensity * float(color[0]) / 256.0, 
            intensity * float(color[1]) / 256.0, 
            intensity * float(color[2]) / 256.0])
        points.append([i * step, tf.alpha_sliders[i].value / 255.0])
        
    btf['colormap']['name'] = 'TransferFunctionEditor'
    btf['colormap']['colors'] = colors
    btf['opacity_curve'] = points
    btf['range'] = [tf.data_range[0], tf.data_range[1]]
    core.set_model_transfer_function(id=fields_model_id, transfer_function=btf)

In [ ]:
tf = TransferFunctionEditor(
    filename=colormap_folder + 'spike_v1.1dt',
    on_change=update_palette)
tf.set_range((0, 5))

### Close-up on contact region

In [ ]:
status = core.set_camera(
    orientation=[0.0, 0.7071067811865475, 0.0, 0.7071067811865476],
    position=[-50.78389040913925, -171.88124847900858, 231.85771136376823],
    target=[-105.20499421405177, -171.88124847900858, 231.85771136376823]
)

### Global view of the spike and receptor

In [ ]:
status = core.set_camera(
    orientation=[0.022513198231254857,  0.04530282427091095,  -0.026285668857608697,  0.9983736142502904],
    position=[-60.794105091613574, -170.9429323911121, 265.85924034226747],
    target=[-73.36804194753815, -164.27604386205962, 125.7347360397887]
)